# Quick tests of different low-pass filters
(c) 2023—24 RTE  
Developed by Grupo AIA

# Summary
This is a (relatively) simple Jupyter Notebook to allow experimenting with the different properties of various second-order low-pass filters, the ones that are available in the dycov tool.

The IEC standard prescribes using a 2nd-order, critically damped filter, presumably because it strikes a good balance between simplicity, frequency rolloff at fc (i.e., how good the filter rejects frequencies above fc) and phase delay properties (i.e., how good the filter preserves the original waveform in the time domain).  But it could be argued that in our case (the dycov tool), it's the last criterion that matters most: we want to filter high frequency "noise" while preserving the waveform as much as possible. It is not always clear which type of filter achieves this. For instance, if we filter by forward-backward sweeping (filtfilt), phase delay gets compensated and it doesn't matter anymore; in that case it's more important that the filter doesn't have "ringing" effects in the step response in the time domain.
Could a Bessel filter be better, then? (in theory it minimizes those ringing effects)

In any case, this Notebook lets you experiment with all these filters and see the effect on test signals.  You may use square wave signals as the most extreme case, the one in which you will see the most differences between filters. But bear in mind that the dycov tool explicitly excludes sharp transtions by means of windowing plus considering exclusion windows.

Maybe a more important setting to play with is how *signal boundaries* are handled, in other words, the **initialization** of the filer.


In [ ]:
import numpy as np
from scipy.signal import freqz
from lp_filters import *
import matplotlib.pyplot as plt

In [ ]:
# Auxiliary functions for plotting
def plot_filtered_signal(signal, filtered_signal, lpf_name):
    plt.figure(figsize=(10, 6))
    plt.plot(t, signal, label="Original", alpha=0.4)
    plt.plot(t, filtered_signal, label="Filtered", linewidth=2)
    plt.title(lpf_name)
    plt.xlabel("Time (s)")
    plt.ylabel("Amplitude")
    plt.legend()
    plt.grid()
    plt.show()


def plot_freq_response(fc, w_ref, h_ref, w, h, filter_name):
    plt.figure(figsize=(8, 6))
    plt.semilogx(w_ref, 20 * np.log10(np.abs(h_ref)), color="silver", ls="dashed")
    plt.semilogx(w, 20 * np.log10(np.abs(h)))
    plt.title(filter_name + " LPF magnitude response (vs. Butterworth)")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude [dB]")
    plt.grid(which="both", axis="both")
    plt.axvline(fc, color="green")  # cutoff frequency
    plt.show()


def plot_phase_response(fc, w_ref, h_ref, w, h, filter_name):
    plt.figure(figsize=(8, 6))
    plt.semilogx(w_ref, np.unwrap(np.angle(h_ref)), color="silver", ls="dashed")
    plt.semilogx(w, np.unwrap(np.angle(h)))
    plt.title(filter_name + " LPF phase response (vs. Butterworth)")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude [dB]")
    plt.grid(which="both", axis="both")
    plt.axvline(fc, color="green")  # cutoff frequency
    plt.axhline(-np.pi / 2, color="red")  # phase midpoint
    plt.show()


def plot_both_responses(fc, w_ref, h_ref, w, h, filter_name):
    plt.figure(figsize=(14, 6))
    # Frequency response
    plt.subplot(1, 2, 1)
    plt.semilogx(w_ref, 20 * np.log10(np.abs(h_ref)), color="silver", ls="dashed")
    plt.semilogx(w, 20 * np.log10(np.abs(h)))
    plt.title(filter_name + " LPF (vs. Butterworth)")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Amplitude [dB]")
    plt.grid(which="both", axis="both")
    plt.axvline(fc, color="green")  # cutoff frequency
    # Phase response
    plt.subplot(1, 2, 2)
    plt.semilogx(w_ref, np.unwrap(np.angle(h_ref)), color="silver", ls="dashed")
    plt.semilogx(w, np.unwrap(np.angle(h)))
    plt.title(filter_name + " LPF (vs. Butterworth)")
    plt.xlabel("Frequency [Hz]")
    plt.ylabel("Phase [rad]")
    plt.grid(which="both", axis="both")
    plt.axvline(fc, color="green")  # cutoff frequency
    plt.axhline(-np.pi / 2, color="red")  # phase midpoint
    plt.show()

# Simple test signal

In [ ]:
# Sampling parameters
fs = 1000  # Sampling frequency in Hz
T = 1  # total time in seconds
t_0 = 0.15  # starting time (use this to test signals that start at non-zero values, etc.)
t = np.linspace(t_0, t_0 + T, T * fs, endpoint=False)  # Time vector (T seconds)

# Generate the test signal
f1, f2 = 5, 50  # Frequencies of the sinusoids in Hz
signal = np.sin(2 * np.pi * f1 * t) + np.sin(2 * np.pi * f2 * t)  # Sum of two sinusoids
# signal = square(2 * np.pi * f1 * t)  # Square wave

# Cutoff frequency in Hz
fc = 15

# Filter the signal

In [ ]:
# Reminder of methods for treating the signal boundaries: "gust", "odd_padding", "even_padding", "constant_padding", "no_padding"

b, a = critically_damped_lpf(fc, fs)
y1 = apply_filtfilt(b, a, signal, method="gust")
plot_filtered_signal(signal, y1, "Critically damped 2nd-order")
print("b =", b, "; a =", a, "\n")

b, a = bessel_lpf(fc, fs)
y2 = apply_filtfilt(b, a, signal, method="gust")
plot_filtered_signal(signal, y2, "Bessel LPF")
print("b =", b, "; a =", a, "\n")

b, a = butter_lpf(fc, fs)
y3 = apply_filtfilt(b, a, signal, method="gust")
plot_filtered_signal(signal, y3, "Butterworth LPF")
print("b =", b, "; a =", a, "\n")

b, a = cheby1_lpf(fc, fs)
y4 = apply_filtfilt(b, a, signal, method="gust")
plot_filtered_signal(signal, y4, "Chebyhev LPF")
print("b =", b, "; a =", a, "\n")

# Optionally, show the filter's characteristics

In [ ]:
if True:
    # To show a frequency range a bit over Nyquist
    worN = np.linspace(0, fs * 0.8, 512)

    # Use Butterworth as the reference to compare against
    b, a = butter_lpf(fc, fs)
    w_ref, h_ref = freqz(b, a, fs=fs, worN=worN)

    # 2nd order critically damped:
    b, a = critically_damped_lpf(fc, fs)
    w, h = freqz(b, a, fs=fs, worN=worN)
    # plot_freq_response(fc, w_ref, h_ref, w, h, "Critically damped 2nd-order")
    # plot_phase_response(fc, w_ref, h_ref, w, h, "Critically damped 2nd-order")
    plot_both_responses(fc, w_ref, h_ref, w, h, "Critically damped 2nd-order")

    # Bessel
    b, a = bessel_lpf(fc, fs)
    w, h = freqz(b, a, fs=fs, worN=worN)
    # plot_freq_response(fc, w_ref, h_ref, w, h, "Bessel")
    # plot_phase_response(fc, w_ref, h_ref, w, h, "Besssel")
    plot_both_responses(fc, w_ref, h_ref, w, h, "Bessel")

    # Chebyshev
    b, a = cheby1_lpf(fc, fs)
    w, h = freqz(b, a, fs=fs, worN=worN)
    # plot_freq_response(fc, w_ref, h_ref, w, h, "Chebyshev")
    # plot_phase_response(fc, w_ref, h_ref, w, h, "Chebyshev")
    plot_both_responses(fc, w_ref, h_ref, w, h, "Chebyshev")